In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm

from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

## 1. Process the Mueller Matrices of the folders

In [2]:
# set the parameters to run the script
data_folder_path = os.path.join(os.getcwd(), r'data')
directories = [data_folder_path]
calib_directory = os.path.join(os.getcwd(), r'calib')

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set

# uncomment to re-process the Mueller matrices
# batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [3]:
time_base = 'T0_'
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path, time_base)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:25<00:00,  2.10s/it]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [4]:
import time

tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    propagation_lists = {}
    
    for tissue_type in tissue_types:
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_lists[tissue_type] = selection_of_ROIs.square_selection(path_folder_temp, path_folder, WM, 
                                                                            tissue_type, path_align_folder)
    # 3. actually do the alignment
    propagation_lists = automatic_ROI_propagation.generate_combined_mask(propagation_lists, path_align_folder)
    automatic_ROI_propagation.do_alignment(path_align_folder)
    output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
    
    # 4. and collect the data after propagation
    for tissue_type in tissue_types:
        WM = tissue_type == 'WM'
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_lists[tissue_type], 
                                                            output_folders, data_folder_path, time_base)

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [11:25<00:00, 114.20s/it]


## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [5]:
from freezingeffect import parameter_comparaison

In [6]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_, max_nb_of_squares = parameter_comparaison.get_parameters()
max_nb_of_squares = max_nb_of_squares * len(base_dirs)

# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

['C:\\Users\\romai\\Documents\\FreezingEffect\\data\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_1',
 'C:\\Users\\romai\\Documents\\FreezingEffect\\data\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_10',
 'C:\\Users\\romai\\Documents\\FreezingEffect\\data\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_11',
 'C:\\Users\\romai\\Documents\\FreezingEffect\\data\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_12',
 'C:\\Users\\romai\\Documents\\FreezingEffect\\data\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_13',
 'C:\\Users\\romai\\Documents\\FreezingEffect\\data\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_5',
 'C:\\Users\\romai\\Documents\\FreezingEffect\\data\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_6',
 'C:\\Users\\romai\\Documents\\FreezingEffect\\data\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_ima

#### 4.2. Get the data for the measurement

In [7]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['WM'][list(data_fixation['WM'].keys())[4]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,136.264151,28.849710,133.483146,136.264151,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.929690,0.011261,0.918987,0.929402,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.027996,0.007756,0.020408,0.027646,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,20.005363,9.413958,13.670886,18.728645,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,28.790958,56.207197,46.516854,28.790958,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.950618,0.013113,0.946835,0.952886,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.034261,0.009053,0.030612,0.033229,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,14.372774,8.709710,12.531646,12.812740,None


In [8]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [9]:
data_fixation['WM'][list(data_fixation['WM'].keys())[0]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,143.994904,26.147991,147.640449,143.994904,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.939922,0.008779,0.932911,0.940156,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.031100,0.007834,0.020408,0.030508,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,34.282007,14.739359,28.481013,32.890347,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,117.196311,43.985927,137.528090,117.196311,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.943044,0.010006,0.932911,0.942537,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.040035,0.008392,0.030612,0.039802,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,15.911063,9.694501,13.670886,14.195352,None


In [10]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [11]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,32.890347,13.680852,13.070828,17.904111,18.728645,11.250481,33.970724,11.263033,11.868027,12.887311,...,16.926175,21.823908,17.633384,15.637837,17.594685,20.396260,12.923779,14.077274,14.058873,11.514054
1,14.195352,13.124163,14.260552,24.287461,12.812740,12.895602,18.712214,14.911444,16.291115,12.730865,...,12.661238,11.314321,13.012232,17.329357,14.182986,19.231722,13.099797,14.948844,15.365850,11.519851


In [12]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
1,0.431596,0.959309,1.091021,1.35653,0.684125,1.146227,0.550834,1.323928,1.372689,0.987861,...,0.748027,0.518437,0.737932,1.108168,0.806095,0.942904,1.01362,1.061913,1.092965,1.000503


#### 4.3. Statistical analysis

In [13]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [14]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z       p_value   n_1   n_2  before/GM  \
parameter      data_type time                                                
azimuth        GM        T0    3200.0  1.000000e+00  80.0  80.0  26.193039   
                         T1    6400.0  1.762654e-31  80.0  80.0  26.193039   
               WM        T0    3528.0  1.000000e+00  84.0  84.0  29.586308   
                         T1    7056.0  5.574848e-33  84.0  84.0  29.586308   
depolarization GM        T0    4050.0  1.000000e+00  90.0  90.0   1.000000   
                         T1    3060.0  6.798259e-02  90.0  80.0   1.000000   
               WM        T0    4050.0  1.000000e+00  90.0  90.0   1.000000   
                         T1    1890.0  8.759415e-10  90.0  84.0   1.000000   
retardance     GM        T0    4050.0  1.000000e+00  90.0  90.0   1.000000   
                         T1    4860.0  2.036292e-05  90.0  80.0   1.000000   
               WM        T0    4050.0  1.000000e+00  90.0  90.0   1.000000   
                         T1    3690.0  7.715222e-01  90.0  84.0   1.000000   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0        23.504169  26.193039     23.504169  
                         T1        23.504169   0.000000      0.000000  
               WM        T0        23.162645  29.586308     23.162645  
                         T1        23.162645   0.000000      0.000000  
depolarization GM        T0         0.000000   1.000000      0.000000  
                         T1         0.000000   1.010034      0.038732  
               WM        T0         0.000000   1.000000      0.000000  
                         T1         0.000000   1.004813      0.013215  
retardance     GM        T0         0.000000   1.000000      0.000000  
                         T1         0.000000   0.819634      0.375274  
               WM        T0         0.000000   1.000000      0.000000  
                         T1         0.000000   0.985391      0.294073

In [15]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [16]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_nb_of_squares)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_nb_of_squares)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_nb_of_squares)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_nb_of_squares)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_nb_of_squares)

retardance_cv

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.00000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.42538,0.278451,1.340026,1.239337,NaN,1.044302,0.605046,0.484705,NaN,0.838356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))